# Automated Project: Planning, Evaluation, Estimation

In [1]:
%pip install crewai crewai-tools

  Using cached crewai-0.108.0-py3-none-any.whl.metadata (33 kB)
  Using cached crewai_tools-0.38.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.8.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached instructor-1.7.7-py3-none-any.whl.metadata (22 kB)
  Using cached json_repair-0.39.1-py3-none-any.whl.metadata (11 kB)
  Using cached json5-0.10.0-py3-none-any.whl.metadata (34 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached litellm-1.60.2-py3-none-any.whl.metadata (36 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pdfplumber-0.11.5-py3-none-any.whl.metadata (42 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cache

## Initiate config and structure

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew, LLM

os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [2]:
# Using local model for development, when complete, change to use OpenAI
# Using 2 model for comparision
ollama_llama_model_name = LLM(model="ollama/llama3.2:3b", base_url="http://localhost:11434", request_timeout=3000)
ollama_qwen_model_name = LLM(model="ollama/qwen2.5:7b", base_url="http://localhost:11434", request_timeout=3000)

files = {
    "agents": "config/automated_project_planning_agents.yaml"
    , "tasks": "config/automated_project_planning_tasks.yaml"
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [3]:
from typing import List 
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class MileStone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[MileStone] = Field(..., description="List of project milestones")

## Initiate agent, task, crew

In [4]:
# Create agents
project_planning_agent = Agent(
    config=agents_config["project_planning_agent"]
    # , llm=ollama_qwen_model_name
)

estimation_agent = Agent(
    config=agents_config["estimation_agent"]
    # , llm=ollama_qwen_model_name
)

resource_allocation_agent = Agent(
    config=agents_config["resource_allocation_agent"]
    # , llm=ollama_qwen_model_name
)

# Create tasks
task_breakdown = Task(
    config=tasks_config["task_breakdown"]
    , agent=project_planning_agent
)

time_resource_estimation = Task(
    config=tasks_config["time_resource_estimation"]
    , agent=estimation_agent
)

resource_allocation = Task(
    config=tasks_config["resource_allocation"]
    , agent=resource_allocation_agent
    , output_pydantic=ProjectPlan
)

# Creating crew
crew = Crew(
    agents=[
        project_planning_agent
        , estimation_agent
        , resource_allocation_agent
    ]
    , tasks=[
        task_breakdown
        , time_resource_estimation
        , resource_allocation
    ]
    , verbose=True
)

## Inputs' for crew

In [5]:
from IPython.display import display, Markdown

project = "Website"
industry = "Technology"
project_objectives = "Create a website for a small business"
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""

display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Run the crew

### Answer from OpenAI model (gpt-4o-mini)

### Answer from Llama3.2:3b

In [6]:
inputs = {
    "project_type": project
    , "project_objectives": project_objectives
    , "industry": industry
    , "team_members": team_members
    , "project_requirements": project_requirements
}

result = crew.kickoff(
    inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 98d92773-77e6-408e-96fd-3accd4d572d2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f4d94384-6bb2-48a9-ba8c-aa9958b5c5a6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f4d94384-6bb2-48a9-ba8c-aa9958b5c5a6
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

🚀 Crew: crew
└── 📋 Task: f4d94384-6bb2-48a9-ba8c-aa9958b5c5a6
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Resource Allocation Strategist
## Final Answer: 
{
  "tasks": [
    {
      "task_name": "Requirement Gathering & Initial Planning",
      "estimated_time_hours": 40,
      "required_resources": ["John Doe"]
    },
    {
      "task_name": "Creating Responsive Design",
      "estimated_time_hours": 80,
      "required_resources": ["Bob Smith"]
    },
    {
      "task_name": "User Interface Development",
      "estimated_time_hours": 40,
      "required_resources": ["Jane Doe"]
    },
    {
      "task_name": "Navigation System Implementation",
      "estimated_time_hours": 60,
      "required_resources": ["Jane Doe"]
    },
    {
      "task_name": "Developing the 'About Us' Page",
      "estimated_time_hours": 20,
      "required_resources": ["Jane Doe"]
    },
    {
      "task_name": "Creating the 'Services' Page",
      "estimated_time_hours": 40,
      "required_resources": ["Jane Doe"]
    },
    {
      "task_name": "Developing the 'Contact Us' Page",
      "estimate

🚀 Crew: crew
├── 📋 Task: f4d94384-6bb2-48a9-ba8c-aa9958b5c5a6
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: d214e39b-dd7b-47e0-a83c-b0747a9225ae
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: 6fdfb59c-ae1a-4880-ad2c-f0d0dfde9c88
       Status: Executing Task...
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f4d94384-6bb2-48a9-ba8c-aa9958b5c5a6
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: d214e39b-dd7b-47e0-a83c-b0747a9225ae
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: 6fdfb59c-ae1a-4880-ad2c-f0d0dfde9c88
       Assigned to: Resource Allocation Strategist
    
       Status: ✅ Completed
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6fdfb59c-ae1a-4880-ad2c-f0d0dfde9c88                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 98d92773-77e6-408e-96fd-3accd4d572d2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Answer from qwen2.5:7b

In [6]:
inputs = {
    "project_type": project
    , "project_objectives": project_objectives
    , "industry": industry
    , "team_members": team_members
    , "project_requirements": project_requirements
}

result = crew.kickoff(
    inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e4ac5a1a-3a77-4b0a-af05-2ac52c96f692                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 41b73809-3e07-4157-90cc-22027d74f5ec
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 41b73809-3e07-4157-90cc-22027d74f5ec
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

🚀 Crew: crew
└── 📋 Task: 41b73809-3e07-4157-90cc-22027d74f5ec
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Resource Allocation Strategist
## Final Answer: 
```json
{
    "tasks": [
        {
            "task_name": "Responsive Design Dev.",
            "action": "Develop responsive design for website",
            "start": 1,
            "end": 2
        },
        {
            "task_name": "User Interface Design",
            "action": "Create user-friendly interface elements",
            "start": 2,
            "end": 3
        },
        {
            "task_name": "Navigation System Dev.",
            "action": "Implement navigation system to ensure easy access to pages",
            "start": 3,
            "end": 4
        },
        {
            "task_name": "About Us Page Creation",
            "action": "Design and develop about us page content",
            "start": 4,
            "end": 5
        },
        {
            "task_name": "Services Page Dev.",
            "action": "Create detailed services page with relevant information",
            "start": 5,
        

🚀 Crew: crew
├── 📋 Task: 41b73809-3e07-4157-90cc-22027d74f5ec
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: ef82d513-9235-44d5-982f-803398ac646e
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: e2d4fc96-9f7a-49f1-b14a-96bde0c52903
       Status: Executing Task...
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed
        ├── 🔧 Using Develop responsive design for website.
        │     - Start: Week 1
        │     - End: Week 2
        │   
        │   - **User Interface Design**:
        │     - Action: Create user-friendly interface elements.
        │     - Start: Week 2
        │     - End: Week 3
        │   
        │   - **Navigation System Dev.**:
        │     - Action: Implement navigation system to ensure easy access to pages.
        │     - Start: Week 3
        │     - End: Week 4
        │   
        │   - **About Us Page Creation**:
        │     - Action: Design and develop about us page content.
        │     - Start: Week 4
        │     - End: Week 5
        │   
        │   - **Services Page Dev.**:
        │     - Action: Create detailed services page with relevant information.
        │     - Start: Week 5
        │     - End: Week 6
        │   
        │   - **Contact Us Page Impl.**:
        │     - Action: Develop contact form and implement contact us functionality.
        │     - Start: Week 6
        │     - End: Week 7
        │   
        │   - **Blog Section Creation**:
        │     - Action: Design blog section layout and develop necessary components.
        │     - Start: Week 7
        │     - End: Week 8
        │   
        │   - **Performance Optimization**:
        │     - Action: Optimize website performance for better user experience.
        │     - Start: Week 9
        │     - End: Week 10
        │   
        │   - **Social Media Integration**:
        │     - Action: Integrate social media platforms to enhance online presence.
        │     - Start: Week 10
        │     - End: Week 11
        │   
        │   - **Testimonials Impl.**:
        │     - Action: Implement testimonials section with customer reviews and feedback.
        │     - Start: Week 11
        │     - End: Week 11
        │   
        │   This format includes the missing " (1)
        ├── 🔧 Using Format the tasks with proper indentation and structure. (2)
        └── 🔧 Using Provide a final organized list of tasks. (1)

🚀 Crew: crew
├── 📋 Task: 41b73809-3e07-4157-90cc-22027d74f5ec
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: ef82d513-9235-44d5-982f-803398ac646e
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: e2d4fc96-9f7a-49f1-b14a-96bde0c52903
       Assigned to: Resource Allocation Strategist
    
       Status: ✅ Completed
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed
        ├── 🔧 Using Develop responsive design for website.
        │     - Start: Week 1
        │     - End: Week 2
        │   
        │   - **User Interface Design**:
        │     - Action: Create user-friendly interface elements.
        │     - Start: Week 2
        │     - End: Week 3
        │   
        │   - **Navigation System Dev.**:
        │     - Action: Implement navigation system to ensure easy access to pages.
        │     - Start: Week 3
        │     - End: Week 4
        │   
        │   - **About Us Page Creation**:
        │     - Action: Design and develop about us page content.
        │     - Start: Week 4
        │     - End: Week 5
        │   
        │   - **Services Page Dev.**:
        │     - Action: Create detailed services page with relevant information.
        │     - Start: Week 5
        │     - End: Week 6
        │   
        │   - **Contact Us Page Impl.**:
        │     - Action: Develop contact form and implement contact us functionality.
        │     - Start: Week 6
        │     - End: Week 7
        │   
        │   - **Blog Section Creation**:
        │     - Action: Design blog section layout and develop necessary components.
        │     - Start: Week 7
        │     - End: Week 8
        │   
        │   - **Performance Optimization**:
        │     - Action: Optimize website performance for better user experience.
        │     - Start: Week 9
        │     - End: Week 10
        │   
        │   - **Social Media Integration**:
        │     - Action: Integrate social media platforms to enhance online presence.
        │     - Start: Week 10
        │     - End: Week 11
        │   
        │   - **Testimonials Impl.**:
        │     - Action: Implement testimonials section with customer reviews and feedback.
        │     - Start: Week 11
        │     - End: Week 11
        │   
        │   This format includes the missing " (1)
        ├── 🔧 Using Format the tasks with proper indentation and structure. (2)
        └── 🔧 Using Provide a final organized list of tasks. (1)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e2d4fc96-9f7a-49f1-b14a-96bde0c52903                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e4ac5a1a-3a77-4b0a-af05-2ac52c96f692                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage Metrics and Costs

In [7]:
import pandas as pd

# $0.150 is cost for OpenAI model, this code just an example
costs = costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0013


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,8933,5311,0,3622,3


In [7]:
result.pydantic.dict

<bound method BaseModel.dict of ProjectPlan(tasks=[TaskEstimate(task_name='Responsive Design Dev.', estimated_time_hours=2.0, required_resources=['Design Tools', 'Web Development Knowledge']), TaskEstimate(task_name='User Interface Design', estimated_time_hours=2.0, required_resources=['UI/UX Design Skills', 'Graphic Design Software']), TaskEstimate(task_name='Navigation System Dev.', estimated_time_hours=2.0, required_resources=['Programming Languages', 'Frontend Development Knowledge']), TaskEstimate(task_name='About Us Page Creation', estimated_time_hours=2.0, required_resources=['Content Writing Skills', 'Website Design Knowledge']), TaskEstimate(task_name='Services Page Dev.', estimated_time_hours=3.0, required_resources=['Service Information Research', 'Web Development Tools']), TaskEstimate(task_name='Contact Us Page Impl.', estimated_time_hours=2.0, required_resources=['Form Implementation Skills', 'Website Backend Knowledge']), TaskEstimate(task_name='Blog Section Creation', e